# Transfer Learning
1. Her veri için tekrardan eğitilmemize gerek kalmadan kendi problemimize uyarlamış oluyoruz.
2. Verilerin bulunduğu ham vektör,bir örneğe ait feature var.Hedef alanı verisi az kaynak verisi de fazladır.
3. Transfer learning yaparsam accuracy daha yüksek olarak başlar.zamandan da kazanç olmuş olur.
4. Negatif transfer riski: transfer automl modellerini kullanarak benzerlik metriklerini yoksa transfer işlemi zararlı hale gelmiş oluyor.
5. Kaynak ve hedef veri farklı olabilir.İstatiksel benzerlikler önemli
6. Transductive (Dönüştürücü) Transfer
7. Denetimsiz Transfer:
8. Fine-tuning önceden eğitilmiş üretilen modeli hafif bir şekilde değiştirmek. son katman atılır. yeni sınıflandırmaya uygun bir yöntem üretilir.
9. Feature çıkartılsın. burası da sınıflandırdığı kısım. Imagenet veri setini kuruyoruz onun sistemini tab kısmı feature kısmını özelliklerini kullanıyoruz.en sonunda sınıflandırma kısmını ayırt ediyorum.modelin ilk %80 ortalama olarak kısmı doldurulur.
10. Feature extraction(özellik çıkarımı):
11. F1 score dengesiz veri setlerinde kullanılır.
12. Mobilenet:keras applications , efficient bir model.Mobil cihazlarda train işlemi yapmıyoruz. Test veya gerçek uygulama şeklinde kullanıyoruz.
13. Multitask learning(Çoklu Görev öğrenme):Çoklu görev öğrenimi sesten görüntüye,metinden görüntüye,
14. NAS sırasında her aday tekrardan eğitilmek yerine supernet eğitiyorum sıfırdan tekrardan aramak varken
13. Eğitilmiş güzel modeller hangi modelin birbirini tamamlaması
14. AUTOML model sayısı belli bu yüzden sürekli öğrenme var.
15. küçük veri seti çıkmazı
16. Few shot: benzerlik sağlayıp önceden öğretilmiş modelin uzayını kullanarak model oluşturmayı da sağlıyor.
17. Prototip ağlar: her sınıfın ortalama vektörü açıklanıyordu. daha sonra benzerlikle bir sınıfa dahil edtiliyordu.
18. Zeo shot:Hiçbir eğitim görmeden direkt sınıflandırma: veri azsa
19. Foundation Modeller: (Temel Modeller) metanın nlp modelleri
20. Sentetik veri ve transfer : çok iyi eğitilmişş gerekiyor. ana modele transfer edilebilir.
21. Data Augmentaion: model işleme anında yapılır. sadece trainde yapılır. validation ve test de yapılmaz diye kaynaklarda geçebilir. Fakat 2024-2025 arasında yeni çıkan kaynaklarda test aşamasında TTE yaklaşımıyla yapılabiliyor.
22. MAML (Model Agnostic ): aşırı hızlı öğrenme
23. LLM-: gelecekte veriler llm aşamasıyla eğitilecek. ve normal model
transformer modeller ve attention mekanizması işin içerisine girdiğinde daha fazla artıyor.
24. Transfer learning en iyi parametrelerle en iyi veriyi buluyor.
H2O:java tabanlı kurumsal firmalar
25. sonra data augmentation inputla veriyi al

In [ ]:
# =========================================
# 0) KURULUM + GLOBAL AYARLAR
# =========================================
"""
UZUN ÖZET (GLOBAL)
------------------
Bu notebook 4 örnekten oluşur ve her örnekte aynı problem üzerinde iki yaklaşımı karşılaştırır:
(1) AutoML: mümkün olduğunca az kodla, model seçimi + hiperparametre arama + ensemble süreçlerini otomatikleştirir.
(2) Transfer Learning (TL): önceden eğitilmiş (pretrained) modelleri alır, yeni probleme uyarlamak için fine-tuning uygular.

Ortak gereksinimler:
- Split stratejisi: %60 train, %20 validation, %20 test
- Süre bütçesi: Varsayılan 5 dk; tek değişkenle artırılabilir: BUDGET_MIN
- Her örnekte skorlar bir tabloda karşılaştırılır.
- Öğrencilerin farklı ekosistemleri görmesi için:
  * Keras Applications (Vision TL) -> 1 kez
  * PyTorch Hub (Vision TL)       -> 1 kez
  * TensorFlow Hub (NLP TL)         -> 1 kez
  * Hugging Face (NLP TL)         -> 1 kez

Not:
- 5 dakikalık demo için veri setleri tfds üzerinden seçildi (kolay erişim, Kaggle kimlik gerektirmez).
- İsterseniz daha sonra aynı şablonu Kaggle veri setlerine (Intel / Cassava Kaggle sürümü / TR sentiment) kolayca uyarlayabiliriz.
"""

# ====== ZAMAN BÜTÇESİ (tek değişken) ======
BUDGET_MIN = 5              # default 5 dakika; artırmak için örn. 15 yapın
RANDOM_SEED = 42

# ====== AutoML bütçeleri ======
AUTOML_TIME_LIMIT = int(BUDGET_MIN * 60)   # saniye

# ====== TL bütçeleri (yaklaşık) ======
# 5 dk => 3-5 epoch; 15 dk => 10-15 epoch gibi ölçekleme
def epochs_from_budget(budget_min: int) -> int:
    if budget_min <= 5:
        return 4
    if budget_min <= 10:
        return 8
    return 12

MAX_EPOCHS = epochs_from_budget(BUDGET_MIN)

# ====== Demo hızlandırma (opsiyonel subsample) ======
# 5 dk modunda çok büyük veri gelirse kısmi örnekleme yapacağız.
FAST_DEMO = True
MAX_TRAIN_SAMPLES = 8000 if FAST_DEMO and BUDGET_MIN <= 5 else None
MAX_VAL_SAMPLES   = 2000 if FAST_DEMO and BUDGET_MIN <= 5 else None
MAX_TEST_SAMPLES  = 2000 if FAST_DEMO and BUDGET_MIN <= 5 else None

# ========== Paket kurulumları ==========
# AutoGluon (tabular & text), AutoKeras (vision AutoML), TabPFN (tabular TL),
# TensorFlow Datasets, TF Hub, Torch, Transformers
!pip -q install -U autogluon.tabular autogluon.multimodal
!pip -q install -U autokeras tensorflow tensorflow-hub tensorflow-datasets
!pip -q install -U torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip -q install -U transformers datasets evaluate scikit-learn tabpfn

import os, time, math, random
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, f1_score, log_loss, roc_auc_score, confusion_matrix
)

np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

def split_60_20_20(X, y, stratify=True):
    """
    60/20/20 split:
      Train = 60%
      Val   = 20%
      Test  = 20%
    """
    strat = y if stratify else None
    X_train, X_temp, y_train, y_temp = train_test_split(
        X, y, test_size=0.40, random_state=RANDOM_SEED, stratify=strat
    )
    strat2 = y_temp if stratify else None
    X_val, X_test, y_val, y_test = train_test_split(
        X_temp, y_temp, test_size=0.50, random_state=RANDOM_SEED, stratify=strat2
    )
    return X_train, X_val, X_test, y_train, y_val, y_test

def maybe_subsample(X, y, max_n):
    if max_n is None or len(X) <= max_n:
        return X, y
    idx = np.random.choice(len(X), size=max_n, replace=False)
    if isinstance(X, pd.DataFrame):
        return X.iloc[idx].reset_index(drop=True), y.iloc[idx].reset_index(drop=True)
    return X[idx], y[idx]

def summarize_binary_metrics(y_true, proba_pos, y_pred=None):
    if y_pred is None:
        y_pred = (proba_pos >= 0.5).astype(int)
    out = {}
    out["Accuracy"] = accuracy_score(y_true, y_pred)
    out["MacroF1"] = f1_score(y_true, y_pred, average="macro")
    try:
        out["ROC-AUC"] = roc_auc_score(y_true, proba_pos)
    except Exception:
        out["ROC-AUC"] = np.nan
    try:
        out["LogLoss"] = log_loss(y_true, np.c_[1-proba_pos, proba_pos], labels=[0,1])
    except Exception:
        out["LogLoss"] = np.nan
    return out

def summarize_multiclass_metrics(y_true, proba, y_pred=None):
    if y_pred is None:
        y_pred = np.argmax(proba, axis=1)
    out = {}
    out["Accuracy"] = accuracy_score(y_true, y_pred)
    out["MacroF1"] = f1_score(y_true, y_pred, average="macro")
    try:
        out["LogLoss"] = log_loss(y_true, proba)
    except Exception:
        out["LogLoss"] = np.nan
    return out

print("BUDGET_MIN:", BUDGET_MIN, "AUTOML_TIME_LIMIT(s):", AUTOML_TIME_LIMIT, "MAX_EPOCHS:", MAX_EPOCHS)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.2/515.2 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.9/98.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.4/74.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.1/452.1 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.8 MB/s eta 0:00:0

ImportError: cannot import name '_yeojohnson_lambda' from 'sklearn.utils.fixes' (/usr/local/lib/python3.12/dist-packages/sklearn/utils/fixes.py)

In [ ]:
# =========================================
# 1) TABULAR (ÖĞRETİCİ) - TITANIC
# =========================================
"""
UZUN ÖZET (Örnek-1: Tabular / Titanic)
-------------------------------------
Amaç:
- Tabular veride AutoML (AutoGluon Tabular) ile TL yaklaşımını (TabPFN) aynı split üzerinde karşılaştırmak.
- Öğrenciler "tabular" veri kavramını, leakage riskini, metrikleri ve otomasyonun rolünü görsün.

Yöntemler:
A) AutoML: AutoGluon Tabular
   - model seçimi + HPO + ensemble/stacking
   - time_limit ile 5 dk default.

B) Transfer Learning (Tabular): TabPFN
   - Tabular için ön-eğitimli (pretrained) bir model yaklaşımı.
   - Küçük/orta tabular veri setlerinde güçlü baseline verir.
   - Bu örnekte TL'nin tabular dünyadaki yerini görselleştirir.

Split:
- %60 train / %20 val / %20 test (stratified)

Metrikler:
- Accuracy, Macro-F1, ROC-AUC, LogLoss

Beklenen gözlem:
- Titanic gibi küçük tabularda AutoML ve TabPFN benzer seviyeye gelebilir;
  AutoML'in avantajı: rapor/leaderboard ve pipeline otomasyonu.
"""

import seaborn as sns
from autogluon.tabular import TabularPredictor
from tabpfn import TabPFNClassifier

df = sns.load_dataset("titanic").dropna(subset=["survived"]).copy()
# Basit bir feature set (öğretici olması için)
use_cols = [
    "survived", "pclass", "sex", "age", "sibsp", "parch", "fare", "embarked", "class", "who", "adult_male", "alone"
]
df = df[use_cols].copy()

target = "survived"
X = df.drop(columns=[target])
y = df[target].astype(int)

X_train, X_val, X_test, y_train, y_val, y_test = split_60_20_20(X, y, stratify=True)

# Demo subsample
X_train, y_train = maybe_subsample(X_train, y_train, MAX_TRAIN_SAMPLES)
X_val, y_val     = maybe_subsample(X_val, y_val, MAX_VAL_SAMPLES)
X_test, y_test   = maybe_subsample(X_test, y_test, MAX_TEST_SAMPLES)

results_1 = []

# ---- A) AutoML: AutoGluon Tabular ----
train_ag = X_train.copy()
train_ag[target] = y_train.values
val_ag = X_val.copy()
val_ag[target] = y_val.values

t0 = time.time()
predictor = TabularPredictor(label=target, eval_metric="log_loss").fit(
    train_data=train_ag,
    tuning_data=val_ag,
    time_limit=AUTOML_TIME_LIMIT,
    presets="medium_quality"
)
t_ag = (time.time() - t0) / 60.0

proba_test = predictor.predict_proba(X_test)  # dataframe with columns [0,1]
proba_pos = proba_test[1].values if 1 in proba_test.columns else proba_test.iloc[:, 1].values
m_ag = summarize_binary_metrics(y_test.values, proba_pos)

results_1.append({
    "Example": "1-Tabular",
    "Method": "AutoML",
    "Model": "AutoGluon Tabular",
    **m_ag,
    "Train_Min": round(t_ag, 2)
})

# ---- B) TL: TabPFN ----
# TabPFN numeric input ister; kategorikleri one-hot ile dönüştürelim (öğretici)
# ---- B) "Transfer Learning" (Tabular için gated olmayan alternatif) ----
# Burada tabular için küçük bir derin model kuruyoruz ve "fine-tune reçetesi" gösteriyoruz.
# Amaç: Öğrencilerin tabularda da representation learning + fine-tuning mantığını görmesi.

import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler

device = "cuda" if torch.cuda.is_available() else "cpu"

# 1) One-hot + ölçekleme
X_train_oh = pd.get_dummies(X_train, dummy_na=True)
X_val_oh   = pd.get_dummies(X_val, dummy_na=True)
X_test_oh  = pd.get_dummies(X_test, dummy_na=True)

X_val_oh  = X_val_oh.reindex(columns=X_train_oh.columns, fill_value=0)
X_test_oh = X_test_oh.reindex(columns=X_train_oh.columns, fill_value=0)

scaler = StandardScaler(with_mean=True, with_std=True)
Xtr = scaler.fit_transform(X_train_oh.values).astype(np.float32)
Xva = scaler.transform(X_val_oh.values).astype(np.float32)
Xte = scaler.transform(X_test_oh.values).astype(np.float32)

ytr = y_train.values.astype(np.int64)
yva = y_val.values.astype(np.int64)
yte = y_test.values.astype(np.int64)

# 2) Mini tabular model (MLP) - hızlı demo
class TabularMLP(nn.Module):
    def __init__(self, d_in: int):
        super(TabularMLP, self).__init__() # Changed this line
        self.net = nn.Sequential(
            nn.Linear(d_in, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(64, 2)
        )
    def forward(self, x):
        return self.net(x)

model = TabularMLP(Xtr.shape[1]).to(device)
criterion = nn.CrossEntropyLoss()

# 3) Eğitim yardımcıları
def batch_iter(X, y, bs=256, shuffle=True):
    idx = np.arange(len(X))
    if shuffle:
        np.random.shuffle(idx)
    for i in range(0, len(X), bs):
        j = idx[i:i+bs]
        yield X[j], y[j]

def eval_proba(model, X):
    model.eval()
    with torch.no_grad():
        logits = model(torch.tensor(X, device=device))
        proba = torch.softmax(logits, dim=1).cpu().numpy()
    return proba

# 4) Strateji-1: Hızlı eğitim (head gibi düşünün) - yüksek LR, az epoch
t0 = time.time()
opt = optim.Adam(model.parameters(), lr=1e-3)
ep1 = max(2, min(5, MAX_EPOCHS))
best_val = -1
best_state = None

for epoch in range(ep1):
    model.train()
    for xb, yb in batch_iter(Xtr, ytr, bs=256, shuffle=True):
        xb_t = torch.tensor(xb, device=device)
        yb_t = torch.tensor(yb, device=device)
        opt.zero_grad()
        loss = criterion(model(xb_t), yb_t)
        loss.backward()
        opt.step()

    # val macro-f1 ile early-stop fikri
    val_proba = eval_proba(model, Xva)
    val_pred = val_proba.argmax(axis=1)
    val_f1 = f1_score(yva, val_pred, average="macro")
    if val_f1 > best_val:
        best_val = val_f1
        best_state = {k: v.detach().cpu().clone() for k,v in model.state_dict().items()}

t_stage1 = (time.time() - t0) / 60.0

# 5) Strateji-2: Fine-tuning - düşük LR ile kısa devam
if best_state is not None:
    model.load_state_dict(best_state)

t0 = time.time()
opt = optim.Adam(model.parameters(), lr=1e-4)  # düşük LR
ep2 = max(1, min(3, MAX_EPOCHS-1))

for epoch in range(ep2):
    model.train()
    for xb, yb in batch_iter(Xtr, ytr, bs=256, shuffle=True):
        xb_t = torch.tensor(xb, device=device)
        yb_t = torch.tensor(yb, device=device)
        opt.zero_grad()
        loss = criterion(model(xb_t), yb_t)
        loss.backward()
        opt.step()

t_stage2 = (time.time() - t0) / 60.0

# 6) Test metrikleri
proba = eval_proba(model, Xte)
proba_pos = proba[:, 1]
m_tl = summarize_binary_metrics(yte, proba_pos)

results_1.append({
    "Example": "1-Tabular",
    "Method": "Transfer",
    "Model": f"TabularMLP (Stage1 ep={ep1}, Stage2 ep={ep2})",
    **m_tl,
    "Train_Min": round(t_stage1 + t_stage2, 2)
})

print("\n=== [AutoGluon] Leaderboard (Val) ===")
lb = predictor.leaderboard(silent=True)
display(lb.head(15))

best_model = predictor.get_model_best()
print("\n=== [AutoGluon] Best Model ===")
print("Best model name:", best_model)

print("\n=== [AutoGluon] Best Model Hyperparameters ===")
# model hyperparameters
try:
    params = predictor.info()["model_info"][best_model]["hyperparameters"]
    print(params)
except Exception as e:
    print("Hyperparameter extraction failed:", e)

print("\n=== [AutoGluon] Best Model Training Info ===")
info = predictor.info()
try:
    model_info = info["model_info"][best_model]
    for k in ["model_type", "fit_time", "predict_time", "val_score"]:
        if k in model_info:
            print(f"{k}: {model_info[k]}")
except Exception as e:
    print("Model info extraction failed:", e)

No path specified. Models will be saved in: "AutogluonModels/ag-20251221_092851"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.5.0
Python Version:     3.12.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Thu Oct  2 10:42:05 UTC 2025
CPU Count:          2
Pytorch Version:    2.9.0+cpu
CUDA Version:       CUDA is not available
Memory Avail:       11.04 GB / 12.67 GB (87.2%)
Disk Space Avail:   83.51 GB / 107.72 GB (77.5%)
Presets specified: ['medium_quality']
Using hyperparameters preset: hyperparameters='default'
Beginning AutoGluon training ... Time limit = 300s
AutoGluon will save models to "/content/AutogluonModels/ag-20251221_092851"
Train Data Rows:    534
Train Data Columns: 11
Tuning Data Rows:    178
Tuning Data Columns: 11
Label Column:       survived
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [np.int

TypeError: TabularMLP.__init__() takes 1 positional argument but 2 were given

In [ ]:
# =========================================
# 2) VISION-1 (GERÇEK DÜNYA) - TF FLOWERS (DÜZELTİLMİŞ)
# =========================================
"""
UZUN ÖZET (Örnek-2: Vision / TF Flowers) - DÜZELTİLMİŞ SÜRÜM
------------------------------------------------------------
Bu örnekte AutoML (AutoKeras) ve Transfer Learning (Keras Applications) aynı veri üzerinde
karşılaştırılır. Split stratejisi %60/%20/%20 olarak uygulanır.

Neden düzeltme gerekti?
- TFDS 'tf_flowers' datasetindeki görüntüler farklı boyutlarda gelir (ör. 320x240, 500x333 vb.).
- Bu yüzden Python listesi -> np.array dönüşümünde NumPy tek bir 4D tensör yapamaz:
  ValueError: inhomogeneous shape.
Çözüm:
- Veriyi önce NumPy'ye çevirmek yerine doğrudan tf.data pipeline üzerinde resimleri resize ederek
  tek tip boyuta getirmek (IMG_SIZE x IMG_SIZE). Böylece hem hata çözülür hem de daha verimli olur.

Kullanılan yöntemler:
A) AutoML: AutoKeras ImageClassifier
B) Transfer Learning: Keras Applications (EfficientNetB0, ImageNet pretrained)
   - Strateji-1: Backbone freeze + head train
   - Strateji-2: Fine-tune (unfreeze) + düşük LR

Metrikler:
- Accuracy, Macro-F1, LogLoss + süre

Bütçe:
- BUDGET_MIN ile kontrol edilir. 5 dk varsayılan.
"""

import tensorflow as tf
import tensorflow_datasets as tfds
import autokeras as ak
import numpy as np
import pandas as pd
import time
from sklearn.metrics import accuracy_score, f1_score, log_loss

tf.random.set_seed(RANDOM_SEED)

# -----------------------------
# 1) Dataset load (tfds)
# -----------------------------
(ds_all, info) = tfds.load("tf_flowers", with_info=True, as_supervised=True)
ds_all = ds_all["train"]

N = info.splits["train"].num_examples  # tf_flowers ~ 3670

# -----------------------------
# 2) %60/%20/%20 split index ile
# -----------------------------
n_train = int(0.60 * N)
n_val   = int(0.20 * N)
n_test  = N - n_train - n_val

# TFDS sıralı geliyor; stratify yok. Eğitim amaçlı kabul edilebilir.
# İsterseniz ileride stratified split için label'ları önce okuyup indeks seçeriz.
ds_train_raw = ds_all.take(n_train)
ds_val_raw   = ds_all.skip(n_train).take(n_val)
ds_test_raw  = ds_all.skip(n_train + n_val).take(n_test)

# -----------------------------
# 3) Preprocess: resize + normalize
# -----------------------------
IMG_SIZE = 160
NUM_CLASSES = info.features["label"].num_classes

def preprocess(img, label):
    img = tf.image.resize(img, (IMG_SIZE, IMG_SIZE))
    img = tf.cast(img, tf.float32) / 255.0
    return img, label

BATCH = 32

train_ds = ds_train_raw.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)\
                      .shuffle(1024, seed=RANDOM_SEED)\
                      .batch(BATCH).prefetch(tf.data.AUTOTUNE)

val_ds   = ds_val_raw.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)\
                    .batch(BATCH).prefetch(tf.data.AUTOTUNE)

test_ds  = ds_test_raw.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)\
                     .batch(BATCH).prefetch(tf.data.AUTOTUNE)

# AutoKeras bazen numpy ister; tf.data ile de çalışabilir ancak bazı sürümlerde sorun çıkabiliyor.
# Bu yüzden "tek tip boyuta gelmiş" veri üzerinden kontrollü şekilde numpy'a geçeceğiz:
def ds_to_numpy(ds, max_batches=None):
    X_list, y_list = [], []
    for i, (xb, yb) in enumerate(ds):
        if max_batches is not None and i >= max_batches:
            break
        X_list.append(xb.numpy())
        y_list.append(yb.numpy())
    X = np.concatenate(X_list, axis=0)
    y = np.concatenate(y_list, axis=0)
    return X, y

# 5 dk demo için opsiyonel kısaltma:
# max_batches = None -> full
max_batches_train = 50 if (FAST_DEMO and BUDGET_MIN <= 5) else None  # ~50*32=1600 örnek
max_batches_val   = 15 if (FAST_DEMO and BUDGET_MIN <= 5) else None
max_batches_test  = 15 if (FAST_DEMO and BUDGET_MIN <= 5) else None

X_train, y_train = ds_to_numpy(train_ds, max_batches=max_batches_train)
X_val,   y_val   = ds_to_numpy(val_ds,   max_batches=max_batches_val)
X_test,  y_test  = ds_to_numpy(test_ds,  max_batches=max_batches_test)

# (AutoKeras için) float32 [0,1] zaten hazır.
results_2 = []

# -----------------------------
# 4) AutoML: AutoKeras
# -----------------------------
max_trials = 10 if BUDGET_MIN <= 5 else 30
ak_epochs  = max(2, min(6, MAX_EPOCHS))

t0 = time.time()
clf = ak.ImageClassifier(overwrite=True, max_trials=max_trials)
clf.fit(
    X_train, y_train,
    epochs=ak_epochs,
    validation_data=(X_val, y_val),
    verbose=1
)
t_ak = (time.time() - t0) / 60.0

model_ak = clf.export_model()
logits = model_ak.predict(X_test, verbose=1)
proba_ak = tf.nn.softmax(logits, axis=1).numpy()
pred_ak = np.argmax(proba_ak, axis=1)

m_ak = {
    "Accuracy": accuracy_score(y_test, pred_ak),
    "MacroF1": f1_score(y_test, pred_ak, average="macro"),
    "LogLoss": log_loss(y_test, proba_ak)
}

print("\n=== [AutoKeras] Best Exported Model Summary ===")
model_ak.summary()

results_2.append({
    "Example": "2-Vision",
    "Method": "AutoML",
    "Model": f"AutoKeras (trials={max_trials}, ep={ak_epochs})",
    **m_ak,
    "Train_Min": round(t_ak, 2)
})

# -----------------------------
# 5) Transfer Learning: Keras Applications (EfficientNetB0)
# -----------------------------
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB0

def build_keras_app_model(trainable_backbone: bool):
    inp = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    x = tf.keras.applications.efficientnet.preprocess_input(inp * 255.0)  # preprocess expects 0-255
    backbone = EfficientNetB0(include_top=False, weights="imagenet", input_tensor=x)
    backbone.trainable = trainable_backbone
    x = layers.GlobalAveragePooling2D()(backbone.output)
    x = layers.Dropout(0.2)(x)
    out = layers.Dense(NUM_CLASSES)(x)
    return models.Model(inp, out)

def eval_keras(model, X, y):
    logits = model.predict(X, verbose=1)
    proba = tf.nn.softmax(logits, axis=1).numpy()
    pred = np.argmax(proba, axis=1)
    return {
        "Accuracy": accuracy_score(y, pred),
        "MacroF1": f1_score(y, pred, average="macro"),
        "LogLoss": log_loss(y, proba)
    }

# Strateji-1: Freeze
tl_ep1 = max(2, min(4, MAX_EPOCHS))
t0 = time.time()
model1 = build_keras_app_model(trainable_backbone=False)
model1.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)
model1.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=tl_ep1, batch_size=32, verbose=1)
t_freeze = (time.time() - t0) / 60.0

m_tl1 = eval_keras(model1, X_test, y_test)

# Model card (reçete)
print("\n=== [Keras TL Recipe] EfficientNetB0 Freeze ===")
print("LR:", 1e-3, "Epochs:", tl_ep1, "Backbone trainable:", False)

results_2.append({
    "Example": "2-Vision",
    "Method": "Transfer",
    "Model": f"KerasApp EffNetB0 (Freeze, ep={tl_ep1}, lr=1e-3)",
    **m_tl1,
    "Train_Min": round(t_freeze, 2)
})

# Strateji-2: Fine-tune (unfreeze) düşük LR
tl_ep2 = max(1, min(3, MAX_EPOCHS-1))
t0 = time.time()
model2 = build_keras_app_model(trainable_backbone=True)
model2.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)
model2.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=tl_ep2, batch_size=32, verbose=1)
t_ft = (time.time() - t0) / 60.0

m_tl2 = eval_keras(model2, X_test, y_test)

print("\n=== [Keras TL Recipe] EfficientNetB0 FineTune ===")
print("LR:", 1e-5, "Epochs:", tl_ep2, "Backbone trainable:", True)

results_2.append({
    "Example": "2-Vision",
    "Method": "Transfer",
    "Model": f"KerasApp EffNetB0 (FineTune, ep={tl_ep2}, lr=1e-5)",
    **m_tl2,
    "Train_Min": round(t_ft, 2)
})

df_res2 = pd.DataFrame(results_2)
display(df_res2)

# -----------------------------
# 6) Ayrıntılı rapor notu (hata ve çözüm)
# -----------------------------
print("\n=== [RAPOR NOTU] Hata ve Çözüm ===")
print("Hata: ValueError: setting an array element with a sequence (inhomogeneous shape)")
print("Neden: tf_flowers görüntüleri farklı boyutlarda geldiği için np.array(images) 4D tensör oluşturamadı.")
print("Çözüm: tf.data pipeline üzerinde preprocess() ile tüm görüntüler (IMG_SIZE, IMG_SIZE) boyutuna resize edildi.")
print("Sonuç: AutoKeras ve Keras Applications eğitimleri tek tip tensörle sorunsuz çalışır.")

ModuleNotFoundError: No module named 'autokeras'